# Agent 
agents refer to systems or components that can autonomously decide what actions to take in a given environment. Agents typically use tools, such as APIs, databases, or retrieval mechanisms, to perform tasks in response to user queries or actions.

# 1. Build an Agent with AgentExecutor (Legacy)

An agent in LangChain refers to a system that leverages a language model (LLM) to decide what actions to take based on input and available tools. Agents are built to perform a sequence of steps in response to user queries, interacting with external resources or tools, and utilizing the LLM to determine which tools to invoke and how to use them.

# Key Concepts of LangChain Agents:

LLM as Reasoning Engine: The language model is used to decide what actions need to be taken based on the input provided. The LLM doesn't perform actions directly; it suggests actions and determines inputs for those actions.

Tools: These are external functions, APIs, or resources that the agent can use. For example, tools can include search engines, databases, or custom functions that the agent can call to gather information or perform tasks.

ction Execution: While the LLM decides on which actions to take, the actions themselves (like calling an API or retrieving data from a local database) are executed by the agent executor, not the LLM itself.

Tool Calls: Tools can be invoked by the agent based on the LLM's reasoning. The agent will pass arguments to the tools based on the query and the context it has, enabling dynamic responses and actions.

# Building an Agent with LangChain
To build an agent in LangChain, you typically:

Define tools (such as a search engine or a custom retriever over a dataset).

Create a language model that can invoke these tools.

Create the agent, which uses the LLM to decide which tools to call.

Use the AgentExecutor to handle the execution of tools based on the agent's decisions.

# Example Walkthrough:

* Define Tools:

Tools could include a search tool like Tavily (for searching the web) or a retriever (for fetching data from a local index).

* Set Up a Language Model:

You can use various LLMs like OpenAI's GPT models or others to guide the agent's reasoning.

* Create the Agent:

The agent is responsible for determining which tool to call based on the user's input.

* Use AgentExecutor:

The AgentExecutor runs the agent, executes the actions, and returns the output.

In [4]:
import getpass
import os

if not os.environ.get("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Tavily API key:\n")

In [8]:
#from langchain.tools import TavilySearchAPIWrapper
from langchain_community.utilities.tavily_search import TavilySearchAPIWrapper


search = TavilySearchAPIWrapper(tavily_api_key="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")


In [12]:
from langchain.agents import Tool


def create_retriever_tool(retriever, tool_name, tool_description):
    # Wrapper function that calls the retriever
    def retriever_function(query: str):
        return retriever.get_relevant_documents(query)

    # Create the tool using the wrapper function
    return Tool(
        name=tool_name,
        func=retriever_function,  # Pass the callable function
        description=tool_description
    )


In [13]:
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Define tools
search = TavilySearchResults(max_results=2)
loader = WebBaseLoader("https://www.accuweather.com/en/in/noida/3146227/daily-weather-forecast/3146227")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
vector = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever_tool = create_retriever_tool(vector.as_retriever(), "search_tool", "Search for relevant documents.")

tools = [search, retriever_tool]

# Define the language model
model = ChatOpenAI(model="gpt-4")

# Create the agent
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")

agent = create_tool_calling_agent(model, tools, prompt)

# Run the agent
agent_executor = AgentExecutor(agent=agent, tools=tools)
response = agent_executor.invoke({"input": "What's the weather in SF?"})

print(response)


{'input': "What's the weather in SF?", 'output': 'The current weather in San Francisco, California is partly cloudy with a temperature of 7.8°C (46.0°F). The wind is coming from NE at a speed of 14.0 kph (8.7 mph). The humidity level is at 63%.'}


# How It Works:
* Tools: The tools (like TavilySearchResults for web searches and a retriever_tool for fetching local data) are defined.
* Agent: The agent decides which tools to invoke based on the user query.
* Executor: The AgentExecutor actually runs the agent and invokes the tools to fetch data, returning the result to the user.

LangChain's agents are useful for building intelligent systems that can perform complex tasks like browsing the web, interacting with databases, and even remembering past interactions (if configured with chat history). They are designed to provide flexibility, enabling agents to dynamically adjust based on the inputs they receive.

# 2. How to migrate from legacy LangChain agents to LangGraph

To migrate from legacy LangChain agents to LangGraph agents, you need to make some adjustments in how you set up and invoke your agent logic. LangGraph's system is more flexible and streamlined compared to LangChain, especially when it comes to agent configuration, memory management, and iteration.

# 1. Basic Setup
In LangChain, agents like AgentExecutor are set up by defining a model, tools, and a prompt template. The AgentExecutor runs the agent with the configured tools and prompt. In LangGraph, this process is simplified, and you use the create_react_agent helper method to set up the agent.

Legacy LangChain AgentExecutor:

In [19]:
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.tools import tool

# Initialize the model
model = ChatOpenAI(model="gpt-4")

# Define the tool function with a docstring
@tool
def magic_function(input: int) -> int:
    """Adds 2 to the input value."""
    return input + 2

# List of tools
tools = [magic_function]

# Define the prompt template (using valid message types)
prompt = ChatPromptTemplate.from_messages(
    [("system", "You are a helpful assistant"),
     ("human", "{input}"),
     ("assistant", "{agent_scratchpad}")  # Using "assistant" instead of "agent"
    ]
)

# Create the agent that will use the tool
agent = create_tool_calling_agent(model, tools, prompt)

# Execute the agent
agent_executor = AgentExecutor(agent=agent, tools=tools)
response = agent_executor.invoke({"input": "what is the value of magic_function(3)?"})

print(response)


{'input': 'what is the value of magic_function(3)?', 'output': 'The value of magic_function(3) is 5.'}


LangGraph Agent Setup:

In [30]:
pip install transformers


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [34]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load the GPT-2 model and tokenizer
model_name = "gpt2"  # Replace with the model name you prefer
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Tokenize input
input_text = "Hello, how are you?"
inputs = tokenizer(input_text, return_tensors="pt")

# Generate a response
outputs = model.generate(inputs['input_ids'], max_length=50)
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(generated_text)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Hello, how are you?

I'm a little bit of a nerd. I'm a big nerd. I'm a big nerd. I'm a big nerd. I'm a big nerd. I'm a big nerd. I'm a


# 2. Handling System Messages
LangChain uses prompt templates with placeholders for the agent's scratchpad. In LangGraph, you can control agent behavior with a state_modifier, which allows you to modify the agent's state before the model is invoked.

Legacy LangChain with Prompt Template:



In [35]:
prompt = ChatPromptTemplate.from_messages(
    [("system", "You are a helpful assistant. Respond only in Spanish."), ("human", "{input}")]
)


LangGraph with System Message:

In [37]:
from langchain.llms import OpenAI
from langchain.agents import create_react_agent

# Initialize OpenAI model (make sure you have access to the API key)
model = OpenAI(model="gpt-3.5-turbo")

# Define your tools (example)
tools = []

# Define system message for the assistant
system_message = "You are a helpful assistant. Respond only in Spanish."

# Create the agent
langgraph_agent_executor = create_react_agent(
    model, tools, state_modifier=system_message
)

# Execute the agent with a query
query = "¿Cómo estás?"
messages = langgraph_agent_executor.invoke({"messages": [("human", query)]})

# Output the response
print(messages["messages"][-1].content)


C:\Users\Admin\AppData\Local\Temp\ipykernel_10856\2390018418.py:5: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  model = OpenAI(model="gpt-3.5-turbo")


TypeError: create_react_agent() got an unexpected keyword argument 'state_modifier'